In [24]:
import pandas as pd

import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [13]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "wine_quality_classification.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "sahideseker/wine-quality-classification",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)



/tmp/ipykernel_9283/1865718131.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


print("Unique quality labels:", df['quality_label'].unique())


quality_order = ["low", "medium", "high"]  
encoder = OrdinalEncoder(
    categories=[quality_order],
    handle_unknown='use_encoded_value',  
    unknown_value=-1  
)
y_encoded = encoder.fit_transform(df[['quality_label']]).ravel()


X = df.drop(columns="quality_label")
y = y_encoded
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  
)


params = {
    "solver": "lbfgs",
    "max_iter": 10000,
    "random_state": 8888,

    "class_weight": "balanced",  
    "penalty": "l2",
    "C": 0.1  
}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)


y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=quality_order))


y_proba = lr.predict_proba(X_test)
print("\nClass probabilities for first 5 samples:")
print(y_proba[:5])

Unique quality labels: ['high' 'medium' 'low']

Accuracy: 0.3800

Classification Report:
              precision    recall  f1-score   support

         low       0.28      0.27      0.27        60
      medium       0.43      0.44      0.43        71
        high       0.41      0.42      0.41        69

    accuracy                           0.38       200
   macro avg       0.37      0.37      0.37       200
weighted avg       0.38      0.38      0.38       200


Class probabilities for first 5 samples:
[[0.3557406  0.3334074  0.31085201]
 [0.35899743 0.29858124 0.34242133]
 [0.33115966 0.33633211 0.33250823]
 [0.34913279 0.3245141  0.32635311]
 [0.36099542 0.30324388 0.3357607 ]]
